Reference: https://huggingface.co/PaddlePaddle/PaddleOCR-VL

In [ ]:
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoProcessor

In [ ]:
import os 
print(os.getenv("CONDA_DEFAULT_ENV"))

In [ ]:
# ---- Settings ----
model_path = "PaddlePaddle/PaddleOCR-VL"
task = "ocr" # Options: 'ocr' | 'table' | 'chart' | 'formula'

In [ ]:
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"

In [ ]:
PROMPTS = {
    "ocr": "OCR:",
    "table": "Table Recognition:",
    "formula": "Formula Recognition:",
    "chart": "Chart Recognition:",
}

### Load the Model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    trust_remote_code=True, 
    dtype=torch.bfloat16,
    device_map = 'cuda:1'
).eval()

processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)

### Load Image

In [ ]:
image_path = "/home/aritrad/test_images/electric.PNG"
image = Image.open(image_path).convert("RGB")

### Create Chat Message

In [ ]:
messages = [
    {
        "role": "user",         
         "content": [
            {
                "type": "image", "image": image
            },
            {
                "type": "text", "text": PROMPTS[task]
            },
        ]
    }
]

In [ ]:
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
inputs = processor(text=[text], images=[image], return_tensors="pt")
inputs = {k: (v.to(DEVICE) if isinstance(v, torch.Tensor) else v) for k, v in inputs.items()}

### Generate OCR Output

In [ ]:
%%time

with torch.inference_mode():
    generated = model.generate(**inputs, max_new_tokens=1024, do_sample=False, use_cache=True)

outputs = processor.batch_decode(generated, skip_special_tokens=True)[0]
print(outputs)